<a href="https://colab.research.google.com/github/mrinal-raj61/Sentiment-Analysis/blob/main/Sentiment_analysis_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
df_raw = pd.read_csv('review_10000.csv')


In [4]:
#assigning score above 3 to 1 and below 3 to -1

df_raw = df_raw[df_raw['Score'] != 4]
df_raw['Sentiment'] = df_raw['Score'].apply(lambda rating :
                                            +1 if rating > 4 else -1)
y = df_raw['Sentiment']


In [5]:
dfNew = df_raw[['Summary','Sentiment']]


In [6]:
#only negative reveiw
dfNew_neg = dfNew[dfNew['Sentiment'] == -1]




In [7]:
#only positive reveiw and number of rows are same as negative reveiws
dfNew_pos =  dfNew[dfNew['Sentiment'] == 1]

dfNew_pos = dfNew_pos.head(len(dfNew_neg.index))



In [8]:
#merge both positive and negative reveiws
df = dfNew_pos.append(dfNew_neg)


DATA CLEANING

In [9]:
def remove_punctuation(text):
    final = "".join(u for u in text if u not in ("?", ".", ";", ":",  "!",'"',",","/" ))
    return final

df = df.dropna(subset=['Summary'])
df['Summary'] = df['Summary'].apply(remove_punctuation)


In [10]:
dfNew2 = df[['Summary','Sentiment']]


In [11]:
#remove stopwords and tokenize
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
from keras.preprocessing.text import Tokenizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [12]:


clean_text = []
for col,row in dfNew2.iterrows():
  text3 = (row[0])
  text = text3.lower()
  text_tokens = word_tokenize(text)
  tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
  text = (" ").join(tokens_without_sw)
  clean_text.append(text)
dfNew2['Statement'] = clean_text


In [13]:
# random split train and test data
import numpy as np
index = df.index
df['random_number'] = np.random.randn(len(index))
train = df[df['random_number'] <= 0.75]
test = df[df['random_number'] > 0.75]


In [14]:
# count vectorizer:
#Create a bag of words
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
train_matrix = vectorizer.fit_transform(train['Summary'])
test_matrix = vectorizer.transform(test['Summary'])



In [15]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [16]:
#Split target and independent variables
X_train = train_matrix
X_test = test_matrix
y_train = train['Sentiment']
y_test = test['Sentiment']

In [17]:
lr.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [18]:
predictions = lr.predict(X_test)



In [19]:
# find accuracy, precision, recall:
from sklearn.metrics import confusion_matrix,classification_report
new = np.asarray(y_test)
confusion_matrix(predictions,y_test)




array([[432, 118],
       [ 57, 433]])

In [31]:


def classifier(text2):

 text = text2.lower()
 new1 = remove_punctuation(text)
 text_tokens = word_tokenize(new1)
 if (text_tokens.count("not") >= 1 ):
  a = text_tokens.count("not")
  fresh_token  = [word for word in text_tokens if not word in stopwords.words()]
  #fresh_token.replace("not"," ")
  fresh_token = [item.replace("not", " ") for item in fresh_token]
  #print(fresh_token)
  text = (" ").join(fresh_token)
  text = [text]
  check_matrix = vectorizer.transform(text)
  X_check = check_matrix
  predicted_value = lr.predict(X_check)
  p = predicted_value * -1**a
  if (p == 1):
    return("Positive Review")
  if (p == -1):
    return("Negative Review")




 
 else:
  fresh_token = [word for word in text_tokens if not word in stopwords.words() ]

  text = (" ").join(fresh_token)
  text = [text]
  check_matrix = vectorizer.transform(text)
  X_check = check_matrix
  predicted_value = lr.predict(X_check)
  if (predicted_value == 1):
    return("Positive Review")
  if (predicted_value == -1):
    return("Negative Review")

  
  





In [35]:
new = ' this is very tasty. I liked it'
print(classifier(new))


Positive Review


In [36]:
new = 'This is the worst food of my life'
print(classifier(new))

Negative Review
